In [1]:
import torch
import numpy
import pandas as pd
import os
import random
import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
model = HookedTransformer.from_pretrained(
    "gpt2-XL",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True
)
#bert-base-cased

Loaded pretrained model gpt2-XL into HookedTransformer


In [74]:
data_path = '../dataset_csvs_singular_plural/s_plurals.csv'
df = pd.read_csv(data_path)
df


,sentence,answer
0,The plural of cat is,cats
1,The plural of dog is,dogs
2,The plural of book is,books
3,The plural of chair is,chairs
4,The plural of house is,houses
...,...,...
122,The plural of place is,places
123,The plural of race is,races
124,The plural of choice is,choices
125,The plural of spice is,spices


In [70]:
#df['sentence'] = df['sentence'].apply(lambda x: ' '.join(x.split()[1:])) #removing the
#df['sentence'] = df['sentence'].apply(lambda x: ' '.join(x.split()[0:2] + x.split()[3:])) #removing of
#df['sentence'] = df['sentence'].apply(lambda x: ' '.join(x.split()[0:4] + x.split()[5:])) #removing is


df['sentence'] = df['sentence'].apply(lambda x: ' '.join(x.split()[0:2] + ['version'] + x.split()[2:])) 
df

,sentence,answer
0,The plural version of cat is,cats
1,The plural version of dog is,dogs
2,The plural version of book is,books
3,The plural version of chair is,chairs
4,The plural version of house is,houses
...,...,...
122,The plural version of place is,places
123,The plural version of race is,races
124,The plural version of choice is,choices
125,The plural version of spice is,spices


In [71]:
probabilities = []
predictions = []
k = 1
#keep_cache = True  # Set this to True if you need to analyze intermediate activations

for i, row in df.iterrows():
    print(i)
    example_prompt = row['sentence']
    example_answer = row['answer']
    tokens = model.to_tokens(example_prompt, prepend_bos=False)
    logits, cache = model.run_with_cache(tokens) #if keep_cache else (model(tokens), None)
    year_probs = torch.softmax(logits[:, -1, :], dim=-1)
    topk = torch.topk(year_probs, k=k)
    topk_tokens = [[model.tokenizer.decode(top) for top in ex] for ex in topk.indices]
    probs_array = topk.values.cpu().detach().numpy()[0].tolist()
    predictions.extend(topk_tokens[0])
    probabilities.extend(probs_array)
    
   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126


In [72]:
sent = []
correct = []
preds = []
probs = []
count = 0
for i, row in df.iterrows():
    example_answer = row['answer']
    pred = predictions[i][1:]
    #print(pred, example_answer)
    example_prompt = row['sentence']
    if example_answer == pred:
        count += 1
        sent.append(example_prompt)
        correct.append(example_answer)
        preds.append(pred)
        probs.append(probabilities[i])
count

6

In [73]:
predictions

[' cats',
 ' "',
 ' "',
 ' chairs',
 ' "',
 ' "',
 ' apples',
 ' the',
 ' "',
 ' cup',
 ' "',
 ' "',
 ' "',
 ' "',
 ' windows',
 ' "',
 ' not',
 ' friend',
 ' "',
 ' teacher',
 ' doctor',
 ' actor',
 ' artist',
 ' athlete',
 ' "',
 ' toy',
 ' "',
 ' "',
 ' pian',
 ' guitar',
 ' viol',
 ' the',
 ' "',
 ' ball',
 ' "',
 ' "',
 ' the',
 ' "',
 ' "',
 ' plate',
 ' "',
 ' fork',
 ' sp',
 ' ring',
 ' "',
 ' skirt',
 ' shoe',
 ' "',
 ' "',
 ' eye',
 ' "',
 ' nose',
 ' the',
 ' "',
 ' toe',
 ' k',
 ' the',
 ' "',
 ' "',
 ' the',
 ' wing',
 ' "',
 ' planets',
 ' called',
 ' "',
 ' "',
 ' sun',
 ' "',
 ' "',
 ' paper',
 ' page',
 ' "',
 ' pencil',
 ' top',
 ' "',
 ' table',
 ' pot',
 ' "',
 ' "',
 ' called',
 ' not',
 ' "',
 ' planes',
 ' "',
 ' "',
 ' "',
 ' "',
 ' "',
 ' nut',
 ' "',
 ' almond',
 ' "',
 ' "',
 ' "',
 ' mountain',
 ' "',
 ' "',
 ' filter',
 ' tire',
 ' rose',
 ' cap',
 ' "',
 ' rope',
 ' "',
 ' "',
 ' click',
 ' "',
 ' image',
 ' set',
 ' "',
 ' "',
 ' "',
 ' "',
 ' "',
 ' boar